In [1]:
# Imports
import pandas as pd
from googleapiclient.discovery import build
from config import api_key

In [2]:
df = pd.read_csv("small_channels.csv")

In [3]:
df.head()

,category,channel,video_url,video_id
0,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0
1,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=NyNF0MfQN8s,NyNF0MfQN8s
2,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=x3vzurNYbyM,x3vzurNYbyM
3,Automobiles,https://www.youtube.com/@AutoAmateur,https://www.youtube.com/watch?v=qysq4YFOwPw,qysq4YFOwPw
4,Automobiles,https://www.youtube.com/@AutoAmateur,https://www.youtube.com/watch?v=B8zgCdINaAs,B8zgCdINaAs


In [4]:
video_ids = df["video_id"].tolist()
video_ids[:5]

['Wqcy9do6YO0', 'NyNF0MfQN8s', 'x3vzurNYbyM', 'qysq4YFOwPw', 'B8zgCdINaAs']

In [5]:
comments = []

In [6]:
# Iterate through video ids
for id in video_ids:

    # Try
    try:

        # empty list for storing reply
        replies = []

        # creating youtube resource object
        youtube = build('youtube', 'v3',
                        developerKey=api_key)

        # retrieve youtube video results
        video_response=youtube.commentThreads().list(
        part='snippet',
        videoId=id
        ).execute()


        # iterate video response
        while video_response:

            # extracting required info
            # from each result object
            for item in video_response['items']:

                # Extracting comments
                comment = item['snippet']['topLevelComment']['snippet']['textOriginal']

                # Store reply is list
                replies.append(comment)

                # print comment with list of reply
                # print(comment, end = '\n\n')

            # Again repeat
            if 'nextPageToken' in video_response and len(replies)<1000:
                video_response = youtube.commentThreads().list(
                        part = 'snippet',
                        videoId = id,
                        pageToken = video_response['nextPageToken']
                    ).execute()
            else:
                comments.append(replies)
                break
    
    # Exception handling
    except Exception as e:
        print(e)
        comments.append(replies)
        pass


In [7]:
len(comments)

108

In [8]:
len(df)

108

In [9]:
df["comments"] = comments
df.head()

,category,channel,video_url,video_id,comments
0,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,[Wheel Cleaner is In production and pre-order ...
1,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=NyNF0MfQN8s,NyNF0MfQN8s,[I know that on my 1122TST it also matters wha...
2,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=x3vzurNYbyM,x3vzurNYbyM,"[Can you put it on top of PPF? Thanks, very ha..."
3,Automobiles,https://www.youtube.com/@AutoAmateur,https://www.youtube.com/watch?v=qysq4YFOwPw,qysq4YFOwPw,[Hey australian Elon Musk can you show a littl...
4,Automobiles,https://www.youtube.com/@AutoAmateur,https://www.youtube.com/watch?v=B8zgCdINaAs,B8zgCdINaAs,[Excellent tutorial video. Congrats! Maybe two...


In [10]:
df.to_csv("small_channels_comments.csv",index=False)
df.to_json("small_channels_comments.json")